In [2]:
import pandas as pd
import numpy as np

#I installed tensorflow for CPU via Anaconda prompt.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.compose import ColumnTransformer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

expectancy = pd.read_csv("life expectancy.csv")
expectancy.head()
expectancy.info()
expectancy.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2938 entries, 0 to 2937
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Country                          2938 non-null   object 
 1   Year                             2938 non-null   int64  
 2   Status                           2938 non-null   object 
 3   Adult Mortality                  2938 non-null   float64
 4   infant deaths                    2938 non-null   int64  
 5   Alcohol                          2938 non-null   float64
 6   percentage expenditure           2938 non-null   float64
 7   Hepatitis B                      2938 non-null   float64
 8   Measles                          2938 non-null   int64  
 9    BMI                             2938 non-null   float64
 10  under-five deaths                2938 non-null   int64  
 11  Polio                            2938 non-null   float64
 12  Total expenditure   

,Year,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling,Life expectancy
count,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2.938000e+03,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000
mean,2007.518720,164.725664,30.303948,4.546875,738.251295,83.022124,2419.592240,38.381178,42.035739,82.617767,5.924098,82.393125,1.742103,6611.523863,1.023085e+07,4.821886,4.852144,0.630362,12.009837,69.234717
std,4.613841,124.086215,117.926501,3.921946,1987.914858,22.996984,11467.272489,19.935375,160.445548,23.367166,2.400770,23.655562,5.077785,13296.603449,5.402242e+07,4.397621,4.485854,0.205140,3.265139,9.509115
min,2000.000000,1.000000,0.000000,0.010000,0.000000,1.000000,0.000000,1.000000,0.000000,3.000000,0.370000,2.000000,0.100000,1.681350,3.400000e+01,0.100000,0.100000,0.000000,0.000000,36.300000
25%,2004.000000,74.000000,0.000000,1.092500,4.685343,82.000000,0.000000,19.400000,0.000000,78.000000,4.370000,78.000000,0.100000,580.486996,4.189172e+05,1.600000,1.600000,0.504250,10.300000,63.200000
50%,2008.000000,144.000000,3.000000,3.755000,64.912906,92.000000,17.000000,43.500000,4.000000,93.000000,5.755000,93.000000,0.100000,1766.947595,1.386542e+06,3.300000,3.300000,0.677000,12.300000,72.100000
75%,2012.000000,227.000000,22.000000,7.390000,441.534144,96.000000,360.250000,56.100000,28.000000,97.000000,7.330000,97.000000,0.800000,4779.405190,4.584371e+06,7.100000,7.200000,0.772000,14.100000,75.600000
max,2015.000000,723.000000,1800.000000,17.870000,19479.911610,99.000000,212183.000000,87.300000,2500.000000,99.000000,17.600000,99.000000,50.600000,119172.741800,1.293859e+09,27.700000,28.600000,0.948000,20.700000,89.000000


In [3]:
#To create a predictive model, knowing from which country data comes can be confusing and it is not a column we can generalize over. 
#We want to learn a general pattern for all the countries, and not only those dependent on specific countries.

expectancy = expectancy.drop(columns = ["Country"])
expectancy.head()

,Year,Status,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,...,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling,Life expectancy
0,2015,Developing,263.0,62,0.01,71.279624,65.0,1154,19.1,83,...,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1,65.0
1,2014,Developing,271.0,64,0.01,73.523582,62.0,492,18.6,86,...,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0,59.9
2,2013,Developing,268.0,66,0.01,73.219243,64.0,430,18.1,89,...,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9,59.9
3,2012,Developing,272.0,69,0.01,78.184215,67.0,2787,17.6,93,...,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8,59.5
4,2011,Developing,275.0,71,0.01,7.097109,68.0,3013,17.2,97,...,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5,59.2


### Splitting data into labels and features.

In [5]:
#Selecting last column.
labels = expectancy.iloc[:,-1]
print(labels.head())

#Selecting all columns excluding the last one.
features = expectancy.iloc[:, 0: -1]
print(features.head())

0    65.0
1    59.9
2    59.9
3    59.5
4    59.2
Name: Life expectancy, dtype: float64
   Year      Status  Adult Mortality  infant deaths  Alcohol  \
0  2015  Developing            263.0             62     0.01   
1  2014  Developing            271.0             64     0.01   
2  2013  Developing            268.0             66     0.01   
3  2012  Developing            272.0             69     0.01   
4  2011  Developing            275.0             71     0.01   

   percentage expenditure  Hepatitis B  Measles    BMI   under-five deaths   \
0               71.279624         65.0      1154   19.1                  83   
1               73.523582         62.0       492   18.6                  86   
2               73.219243         64.0       430   18.1                  89   
3               78.184215         67.0      2787   17.6                  93   
4                7.097109         68.0      3013   17.2                  97   

   Polio  Total expenditure  Diphtheria    HIV/AIDS 

### One-hot-encoding of categorical data.

In [6]:
features = pd.get_dummies(features)
print(features)

      Year  Adult Mortality  infant deaths  Alcohol  percentage expenditure  \
0     2015            263.0             62     0.01               71.279624   
1     2014            271.0             64     0.01               73.523582   
2     2013            268.0             66     0.01               73.219243   
3     2012            272.0             69     0.01               78.184215   
4     2011            275.0             71     0.01                7.097109   
...    ...              ...            ...      ...                     ...   
2933  2004            723.0             27     4.36                0.000000   
2934  2003            715.0             26     4.06                0.000000   
2935  2002             73.0             25     4.43                0.000000   
2936  2001            686.0             25     1.72                0.000000   
2937  2000            665.0             24     1.68                0.000000   

      Hepatitis B  Measles    BMI   under-five deat

### Train-test split

In [19]:
feat_train, feat_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.3, random_state = 42)

print(feat_train.head())
print(labels_train)
print(labels_test)

      Year  Adult Mortality  infant deaths  Alcohol  percentage expenditure  \
1641  2008             64.0              0     7.14             2655.573684   
618   2005            394.0              8     2.03                0.000000   
406   2009            283.0             44     4.55               81.143047   
1094  2011            289.0              4     3.57               40.453674   
1788  2007            217.0             58     0.26                0.530573   

      Hepatitis B  Measles    BMI   under-five deaths   Polio  ...  \
1641         86.0         1   66.6                   0   72.0  ...   
618          92.0       146   21.7                  13   62.0  ...   
406          92.0     54118   16.1                  77   91.0  ...   
1094         86.0         0   23.7                   7   85.0  ...   
1788         85.0      1088   17.6                  78   84.0  ...   

      Diphtheria    HIV/AIDS           GDP  Population   thinness  1-19 years  \
1641         72.0      

### Extracting float and integer variables and assigning them to variable with columns. It is required for ColumnTransformer.

In [8]:
num_feats = features.select_dtypes(include = ['float64', 'int64'])
num_cols = num_feats.columns

print(num_cols)

Index(['Year', 'Adult Mortality', 'infant deaths', 'Alcohol',
       'percentage expenditure', 'Hepatitis B', 'Measles ', ' BMI ',
       'under-five deaths ', 'Polio', 'Total expenditure', 'Diphtheria ',
       ' HIV/AIDS', 'GDP', 'Population', ' thinness  1-19 years',
       ' thinness 5-9 years', 'Income composition of resources', 'Schooling'],
      dtype='object')


### Standarization

In [22]:
#Definig ColumnTransformer model 
ct = ColumnTransformer([('only_numeric', StandardScaler(), num_cols)], remainder = 'passthrough')

#Fitting and transforming trainning data in the model.
feat_train_scaled = ct.fit_transform(feat_train)
feat_test_scaled = ct.transform(feat_test)

print(feat_train_scaled, feat_test_scaled)

[[ 0.09628979 -0.81092833 -0.25772413 ...  0.78649403  1.
   0.        ]
 [-0.55350825  1.87482792 -0.18585883 ... -0.82511401  0.
   1.        ]
 [ 0.31288913  0.97143718  0.13753501 ... -1.9098502   0.
   1.        ]
 ...
 [-0.12030956  0.83308004 -0.10501037 ... -1.13503864  0.
   1.        ]
 [-0.98670694 -0.74581909 -0.23975781 ...  1.03443373  1.
   0.        ]
 [-0.55350825 -1.05508799 -0.19484199 ... -2.06481251  0.
   1.        ]] [[-0.3369089  -0.33074767 -0.18585883 ... -0.20526477  0.
   1.        ]
 [-0.3369089  -0.41213422 -0.25772413 ...  0.56954679  1.
   0.        ]
 [-0.12030956 -0.31447036 -0.25772413 ...  0.47656941  0.
   1.        ]
 ...
 [-0.77010759 -0.77023506 -0.24874097 ...  2.67703424  1.
   0.        ]
 [ 0.74608782 -0.85976026 -0.25772413 ...  1.2203885   1.
   0.        ]
 [-0.98670694  0.87377332 -0.23975781 ... -0.48419693  0.
   1.        ]]


### Building the model

Here, we are implementing model with one hidden layer with 128 neurons, ReLu as activation function and adam optimalizer.

In [9]:
#Implemening sequential model
model = Sequential()
#Defining input with shape adjusted to features.
input = InputLayer(input_shape = (features.shape[1]))
#Adding input layer
model.add(input)
#Adding hidden layer
model.add(Dense(128, activation = 'relu'))
#Adding output layer
model.add(Dense(1))
print(model)

In [10]:
#Defininig optimizer.
opt = Adam(learning_rate = 0.01)
print(opt)

In [11]:
#Model compilation.
model.compile(loss = 'mse', metrics = ['mae'], optimizer = opt)

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               2816      
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 2,945
Trainable params: 2,945
Non-trainable params: 0
_________________________________________________________________
None


### Fitting and evaluating the model.

In [26]:
#For now, I'm struggling with "ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float)."
#Fitting the model.
model.fit(feat_train_scaled, labels_train, epochs = 40, batch_size = 1, verbose = 1)

#Evaluating model's metrics and loss function.
res_mse, res_mae = model.evaluate(feat_test_scaled, labels_test)

print(res_mse, res_mae)

Epoch 1/40
2056/2056 [==============================] - 4s 2ms/step - loss: 24.0580 - mae: 3.3417
Epoch 2/40
2056/2056 [==============================] - 4s 2ms/step - loss: 11.5645 - mae: 2.4541
Epoch 3/40
2056/2056 [==============================] - 4s 2ms/step - loss: 11.1311 - mae: 2.2756
Epoch 4/40
2056/2056 [==============================] - 4s 2ms/step - loss: 9.1661 - mae: 2.1824
Epoch 5/40
2056/2056 [==============================] - 3s 2ms/step - loss: 10.3321 - mae: 2.1775
Epoch 6/40
2056/2056 [==============================] - 3s 2ms/step - loss: 8.9709 - mae: 2.0994
Epoch 7/40
2056/2056 [==============================] - 4s 2ms/step - loss: 8.4136 - mae: 2.0230
Epoch 8/40
2056/2056 [==============================] - 4s 2ms/step - loss: 7.3045 - mae: 1.9498
Epoch 9/40
2056/2056 [==============================] - 4s 2ms/step - loss: 7.9184 - mae: 2.0010
Epoch 10/40
2056/2056 [==============================] - 4s 2ms/step - loss: 6.9898 - mae: 1.9120
Epoch 11/40
2056/2056 [==